# Introduction

This notebook gives some examples on how to use the new function calling API by OpenAI. See [here](https://platform.openai.com/docs/guides/function-calling) for more details on functions

In [7]:
import os

import dotenv
from openai import AzureOpenAI

dotenv.load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version = "2023-07-01-preview",
    api_key = os.environ.get("AZURE_OPENAI_API_KEY"),
)

In [8]:
def get_chat_completion(messages: list[dict], functions: list[dict], *args, **kwargs):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=0.7,
        max_tokens=2_000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        functions=functions,
        stop=None,
        stream=False,
        *args,
        **kwargs
    )
    return response


In [10]:
functions = [
    {
        "name": "lookup_weather",
        "description": "Find the weather for a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to look the weather for"
                },
            },
            "required": ["city"]
        }
    }
]

messages = [
    {
        "role": "user",
        "content": "What's the weather in Washington, D.C.?"

    }
]
response = get_chat_completion(
    messages=messages,
    functions=functions,
    function_call="auto")

In [5]:
response.choices


[<OpenAIObject at 0x21ea8a559e0> JSON: {
   "index": 0,
   "message": {
     "role": "assistant",
     "content": null,
     "function_call": {
       "name": "lookup_weather",
       "arguments": "{\n  \"city\": \"Washington, D.C.\"\n}"
     }
   },
   "finish_reason": "function_call"
 }]